In [23]:
"""
lstm_forecast_per_category.py

- Trains a univariate LSTM per Product_Category using historical Units_Sold
- Uses a sliding window (SEQ_LEN) from the series as input
- Evaluates on a temporal holdout (last TEST_WINDOW_DAYS)
- Produces iterative HORIZON_DAYS forecasts per category
- Saves each category model + scaler and writes final CSV of forecasts

Requirements:
    pip install pandas numpy scikit-learn tensorflow joblib
"""

import os
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# -------------- USER CONFIG --------------
DATA_PATH = "store_sales_forecasting_cleaned_new.csv"  # update to your filename
OUT_DIR = "lstm_out"
SEQ_LEN = 30                # lookback window (days)
TEST_WINDOW_DAYS = 14       # last N days reserved for temporal holdout evaluation
HORIZON_DAYS = 7            # forecast horizon (days)
EPOCHS = 50
BATCH_SIZE = 32
RND_SEED = 42
TARGET_NAMES = ["Units_Sold","Units Sold","units_sold","Units","Quantity","Qty","units"]
CAT_COL = "Product_Category"
DATE_COL = "Date"
# -----------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)
np.random.seed(RND_SEED)
tf.random.set_seed(RND_SEED)

# ---------- helpers ----------
def detect_target(df):
    for t in TARGET_NAMES:
        if t in df.columns:
            return t
    # try case-insensitive
    cols_lc = {c.lower(): c for c in df.columns}
    for t in TARGET_NAMES:
        if t.lower() in cols_lc:
            return cols_lc[t.lower()]
    raise KeyError(f"No target column found. Expected one of {TARGET_NAMES}. File columns: {list(df.columns)}")

def create_sequences(values, seq_len):
    X, y = [], []
    for i in range(len(values) - seq_len):
        X.append(values[i:i+seq_len])
        y.append(values[i+seq_len])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    return X, y

# ---------- load and basic clean ----------
df = pd.read_csv(DATA_PATH)
# parse date robustly (day-first common in your files)
df[DATE_COL] = pd.to_datetime(df[DATE_COL].astype(str).str.strip(), dayfirst=True, errors='coerce')
df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL).reset_index(drop=True)

TARGET = detect_target(df)
print("Using target column:", TARGET)

if CAT_COL not in df.columns:
    raise KeyError(f"Category column '{CAT_COL}' not found in CSV.")

# ---------- train per-category LSTM ----------
categories = df[CAT_COL].unique().tolist()
all_forecasts = []

for cat in categories:
    print("\n=== CATEGORY:", cat, "===")
    sub = df[df[CAT_COL] == cat].sort_values(DATE_COL).reset_index(drop=True).copy()
    sub = sub.dropna(subset=[TARGET])
    sub[TARGET] = sub[TARGET].astype(float)

    if len(sub) < (SEQ_LEN + 10):
        print(f"  Not enough rows ({len(sub)}) for seq_len={SEQ_LEN}. Skipping.")
        continue

    # temporal split
    last_date = sub[DATE_COL].max()
    test_start = last_date - pd.Timedelta(days=TEST_WINDOW_DAYS - 1)

    train_series = sub[sub[DATE_COL] < test_start][TARGET].values
    test_series = sub[sub[DATE_COL] >= test_start][TARGET].values
    full_series = sub[TARGET].values  # full history for iterative forecast

    # scale on train only
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(train_series.reshape(-1,1))
    train_scaled = scaler.transform(train_series.reshape(-1,1)).flatten()

    # create sequences
    X_train, y_train = create_sequences(train_scaled, SEQ_LEN)

    # small validation split
    val_split = 0.1 if X_train.shape[0] >= 50 else 0.2

    # build model
    model = Sequential([
        LSTM(64, input_shape=(SEQ_LEN,1), return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    # callbacks
    model_path = os.path.join(OUT_DIR, f"{cat}_best.h5")
    es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
    mc = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=0)

    # train
    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=val_split,
        callbacks=[es, mc],
        verbose=1
    )

    # save scaler + model final
    joblib.dump(scaler, os.path.join(OUT_DIR, f"{cat}_scaler.pkl"))
    model.save(os.path.join(OUT_DIR, f"{cat}_final.h5"))
    print(f"  Saved model & scaler to {OUT_DIR}")

    # Evaluate on test by creating sliding sequences that cross into test region
    combined = np.concatenate([train_series, test_series])
    combined_scaled = scaler.transform(combined.reshape(-1,1)).flatten()
    X_test_seq, y_test_true = [], []
    start_idx = max(0, len(train_series) - SEQ_LEN)
    for i in range(start_idx, len(combined) - SEQ_LEN):
        X_test_seq.append(combined_scaled[i:i+SEQ_LEN])
        y_test_true.append(combined[i+SEQ_LEN])
    if len(X_test_seq) > 0:
        X_test_seq = np.array(X_test_seq).reshape((-1, SEQ_LEN, 1))
        y_test_true = np.array(y_test_true)
        preds_scaled = model.predict(X_test_seq).flatten()
        preds = scaler.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
        mae = mean_absolute_error(y_test_true, preds)
        rmse = mean_squared_error(y_test_true, preds)
        mape = np.mean(np.abs((y_test_true - preds) / (y_test_true + 1e-9))) * 100
        print(f"  Test MAE: {mae:.3f}, RMSE: {rmse:.3f}, MAPE: {mape:.3f}%")
    else:
        print("  Not enough overlap for test evaluation; skipping evaluation.")

    # iterative 7-day forecast using full history
    history_values = full_series.copy().tolist()
    preds_cat = []
    for k in range(HORIZON_DAYS):
        seq = np.array(history_values[-SEQ_LEN:]).reshape(-1,1)
        seq_scaled = scaler.transform(seq).flatten()
        X_pred = seq_scaled.reshape((1, SEQ_LEN, 1))
        yhat_scaled = model.predict(X_pred).flatten()[0]
        yhat = scaler.inverse_transform(np.array([[yhat_scaled]]))[0,0]
        preds_cat.append(yhat)
        history_values.append(yhat)

    future_dates = [last_date + timedelta(days=i) for i in range(1, HORIZON_DAYS+1)]
    for dt, val in zip(future_dates, preds_cat):
        all_forecasts.append({'Date': dt, CAT_COL: cat, 'Pred_Units_Sold': float(val)})

# ---------- save results ----------
pred_df = pd.DataFrame(all_forecasts)
if pred_df.empty:
    print("No forecasts generated. Check data sufficiency.")
else:
    pivot = pred_df.pivot(index='Date', columns=CAT_COL, values='Pred_Units_Sold').reset_index()
    pivot['total_units'] = pivot.drop(columns='Date').sum(axis=1)
    out_path = os.path.join(OUT_DIR, "lstm_7day_forecast_by_category.csv")
    pivot.to_csv(out_path, index=False)
    print("\nSaved LSTM 7-day forecast to:", out_path)
    print(pivot.head(HORIZON_DAYS))


Using target column: Units_Sold

=== CATEGORY: 0 ===
Epoch 1/50
17/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0031 - mae: 0.0403

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0031 - mae: 0.0389 - val_loss: 0.0013 - val_mae: 0.0250
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018 - mae: 0.0236

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0019 - mae: 0.0237 - val_loss: 9.7382e-04 - val_mae: 0.0241
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0018 - mae: 0.0233 - val_loss: 9.8553e-04 - val_mae: 0.0240
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0018 - mae: 0.0235 - val_loss: 0.0010 - val_mae: 0.0239
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0017 - mae: 0.0235 - val_loss: 0.0010 - val_mae: 0.0239
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0017 - mae: 0.0233 - val_loss: 0.0010 - val_mae: 0.0238
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0017 - mae: 0.0230 - val_loss: 0.0010 - val_mae: 0.0238
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0017 - mae: 0.0231 - val_loss: 0.0010 - val_mae: 0.0237
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 2.


  Saved model & scaler to lstm_out
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
  Test MAE: 5.690, RMSE: 43.828, MAPE: 8.700%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

=== CATEGORY: 2 ===
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016 - mae: 0.0267

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0016 - mae: 0.0265 - val_loss: 3.2985e-04 - val_mae: 0.0148
Epoch 2/50
11/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.8071e-04 - mae: 0.0191 

...

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0124 - mae: 0.0807 - val_loss: 0.0123 - val_mae: 0.0785
Epoch 19/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0131 - mae: 0.0820 - val_loss: 0.0125 - val_mae: 0.0789
Epoch 20/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0122 - mae: 0.0802

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0121 - mae: 0.0802 - val_loss: 0.0122 - val_mae: 0.0784
Epoch 21/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0121 - mae: 0.0805 - val_loss: 0.0125 - val_mae: 0.0783
Epoch 22/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0122 - mae: 0.0803 - val_loss: 0.0123 - val_mae: 0.0778
Epoch 23/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0121 - mae: 0.0796 - val_loss: 0.0123 - val_mae: 0.0779
Epoch 24/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0123 - mae: 0.0797 - val_loss: 0.0124 - val_mae: 0.0781
Epoch 25/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0122 - mae: 0.0784

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0121 - mae: 0.0784 - val_loss: 0.0121 - val_mae: 0.0778
Epoch 26/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0122 - mae: 0.0794 - val_loss: 0.0122 - val_mae: 0.0773
Epoch 27/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0117 - mae: 0.0798 - val_loss: 0.0123 - val_mae: 0.0775
Epoch 28/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0123 - mae: 0.0803 - val_loss: 0.0123 - val_mae: 0.0772
Epoch 29/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0126 - mae: 0.0815

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0126 - mae: 0.0814 - val_loss: 0.0118 - val_mae: 0.0772
Epoch 30/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0118 - mae: 0.0800 - val_loss: 0.0123 - val_mae: 0.0771
Epoch 31/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0115 - mae: 0.0780 - val_loss: 0.0127 - val_mae: 0.0773
Epoch 32/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0118 - mae: 0.0779 - val_loss: 0.0122 - val_mae: 0.0768
Epoch 33/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0121 - mae: 0.0788 - val_loss: 0.0119 - val_mae: 0.0769
Epoch 34/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0123 - mae: 0.0812 - val_loss: 0.0120 - val_mae: 0.0768
Epoch 35/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0114 - mae: 0.0772 - val_loss: 0.0119 - val_mae: 0.0767
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 29.


  Saved model & scaler to lstm_out
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
  Test MAE: 5.797, RMSE: 66.378, MAPE: 15.995%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Saved LSTM 7-day forecast to: lstm_out/lstm_7day_forecast_by_category.csv
Product_Category       Date          0          1           2  total_units
0                2025-12-13  64.059563  39.325989  108.077087   211.462639
1                2025-12-14  64.051582  45.130676  115.484825   224.667084
2                2025-12-15  64.041199  48.390911  120.288803   232.720913
3                2025-12-16  64.031914  48.160152  119.907021   232.099087
4                2025-12-17  64.024300  44.476101  114.298599   222.799000
5                2025-12-18  64.018776  38.297073  105.160355   207.476204
6           